# Inspecting a LAS/LAZ file

In [ ]:
pdal --version

In [ ]:
ls -l ./data

In general, it is best to use absolute paths to specify paths to datafiles.  Relative paths (e.g. './data/file.laz', '$PWD/data/file.laz') may cause issues.

In [ ]:
pwd

In [ ]:
laz_file=$PWD'/data/OR_WizardIsland.laz'
CWD=`pwd`
echo $laz_file
echo $CWD

## Metadata

In [ ]:
pdal info --metadata $laz_file

To get the schema for a dataset:

In [ ]:
pdal info --schema ./data/FoxIsland.laz

## Printing Points

Print the values for the first point in the file:

In [ ]:
pdal info $laz_file -p 0

Print the values for the first 5 points in the file

In [ ]:
pdal info $laz_file -p 0-10

Print the first 10 points, and extract only the elevation values using bash commands

In [ ]:
pdal info $laz_file -p 0-10|grep  "Z"|awk '{print $2}' FS=': '

jq is a command line JSON parser, and can be a useful tool for these types of operations. Note if returning more than one point, specify "[]" to return all the points in the array.

In [ ]:
pdal info $laz_file -p 0-10|jq -r .points.point[].Z

## Classifications

Use the [info application](https://pdal.io/en/2.4.3/apps/info.html) with the --stats flag and perform filtering to get a summary of the classifications for a given lidar file.

In [ ]:
pdal info ./data/OR_WizardIsland.laz --stats --filters.stats.dimensions=Classification  --filters.stats.count=Classification

Perform both a metadata dump and classification count and output to a JSON file:

In [ ]:
pdal info --metadata --stats --filters.stats.dimensions=ReturnNumber,Classification --filters.stats.count=ReturnNumber,Classification ./data/OR_WizardIsland.laz > ./data/fileinfo.json

## Reprojection

In [ ]:
cd pipelines

In [ ]:
ls

Run a pipeline to project a file from Oregon Lambert to Geographic Lat/Lon coordindates:

In [ ]:
pdal pipeline Reproject_Ex.json

In [ ]:
cd ../data
ls

Inspect the coordinates to make sure the xy values make sense..

In [ ]:
pdal info --metadata OR_WizardIsland_LatLon.laz

# Tiling data

In [37]:
pdal pipeline ./pipelines/TileLAZ.json

## Calculating a Boundary

Utilizing the info command, the boundary of a dataset can be obtained by using the "--boundary" flag.  This will output the boundary in WKT format in JSON-formatted output.

In [ ]:
ls

Use the "--boundary" flag to get the boundary of data over the Siuslaw River:

In [ ]:
pdal info ./data/Siuslaw.laz --boundary 

However, to get a boundary in vector format to visualize in a GIS or Google Earth requires some additional steps.  The PDAL command, [tindex](https://pdal.io/en/2.5.3/apps/tindex.html#tindex-command) is used to create a boundary that utilizes the [hexbin filter](https://pdal.io/en/2.5.3/stages/filters.hexbin.html#filters-hexbin)

To get a basic boundary:

In [ ]:
pdal tindex create --tindex ./data/Siuslaw_bounds.shp --filespec ./data/Siuslaw.laz -f "ESRI Shapefile"

In [ ]:
cd $CWD

Load the shapefile into a GIS to see its extent:
![Siuslaw Bounds](./images/SiuslawRiverBounds.jpeg)

For rough estimations of boundaries, this is usually sufficient. To obtain a more precise fit of the data alter some of the parameters in the [filters.hexbin](https://pdal.io/en/2.5.3/stages/filters.hexbin.html#filters-hexbin) command.  The "edge_size" parameter is particularly useful for this scenario as it controls the size of the hexagon boundaries used to estimate whether a section of the dataset should be considered. Finding an appropriate value for edge_size can be an iterative process.  For example, try using a vlue of "50" units (for this dataset, the units are feet).

In [ ]:
pdal tindex create --tindex ./data/Siuslaw_bounds50.shp --filters.hexbin.edge_size=50 --filespec ./data/Siuslaw.laz -f "ESRI Shapefile"

In [ ]:
cd $CWD

Load this version of the boundary into a GIS:
![Siuslaw Bounds_Fit](./images/SiuslawRiverBounds_Fit.png)

Note how this is a much better fit to the data, and shows regions where there is no data (over the water). However, use caution with the edge_size parameter as setting it too low might not capture an appropriate amount of data, and mis-represent the data coverage.

## Filtering Noise

In [ ]:
pdal pipeline ./pipelines/NoiseFilter.json

In [ ]:
pdal info ./data/FoxIsland_Clean.laz --stats --filters.stats.dimensions=Classification  --filters.stats.count=Classification

# Thinning Data

Point cloud files can often be quite large and cumbersome to work with.  Depending on the objective, it is often useful to thin a dataset in order to make it easier and faster to work with. The [filters.sample](https://pdal.io/en/2.5.3/stages/filters.sample.html#filters-sample) utilizes a Poisson sampling to thin the dataset.

In [ ]:
pdal translate ./data/FoxIsland_Clean.laz ./data/FoxIsland_Clean_Thin1m.laz sample --filters.sample.radius=1

# Ground Classifications

In [ ]:
pdal pipeline ./pipelines/ExtractGround.json

In [ ]:
pdal pipeline ./pipelines/CreateGround.json

# Create Rasters

In [ ]:
pdal pipeline ./pipelines/CreateDSM.json

In [ ]:
pdal pipeline ./pipelines/CreateDTM.json